### 동아 일보 데이터크롤러

```
Arguments : query_word, (period)

return : title_list, category_list, body_list
```

In [2]:
import requests
from bs4 import BeautifulSoup

In [194]:
def body_extractor(link):
    soup = BeautifulSoup(requests.get(link).content, 'html.parser')
    
    text = ""    
    for con in soup.select_one(".article_txt").contents:
        try:
            con.text
            continue
        except :
            if con == "\ufeff【서울=뉴시스】":
                continue
            text += " " + con
    return text.strip()

def doonga_crawler(query, period=None):
    
    # data containers 
    titles = []
    links = []
    categories = []
    dates = []
    bodies = []
    
    i = 1 
    nobody = 0
    
    while True:
        print("-" * 30)
        print('{} page is start'.format(i//15 + 1))
        
        url = "http://news.donga.com/search?p={}&query={}&check_news=1&more=1&sorting=1&search_date=1&v1=&v2=&range=1".format(i, query)
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'html.parser')
        
        if len(soup.select('p.tit')) == 0:
            print('***** no result! *****')
            break

        for art, cate, da in zip(soup.select("p.tit > a"), soup.select(".loc"), soup.select("p.tit > span")):
            
            title = art.text
            if len(title) == 0:
                nobody += 1
                continue
                
            link = art["href"]
            if len(link) == 0:
                nobody += 1
                continue
                
            category = cate.text
            if len(category) == 0:
                category = 'unknown_category'
                
            date = da.text
            if len(date) == 0:
                date = "unknown_date"
            
            try:
                body = body_extractor(link)
                if len(body) == 0:
                    print("no body")
                    nobody += 1
                    continue
                    
            except Exception as ex:
                print(ex)
                nobody += 1
                continue
                
            titles.append(title)
            links.append(link)
            categories.append(category) 
            dates.append(date)
            bodies.append(body)
            
        print('{} page is done'.format(i//15 + 1))
        i += 15
    return [titles, links, categories, dates, bodies], nobody

In [195]:
data_dust, nobody_dust = doonga_crawler("미세먼지", period=None)

------------------------------
1 page is start
1 page is done
